In [1]:
# Import Dependencies and modules
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
from spacy.lang.en import English
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from string import punctuation
from collections import Counter
from io import StringIO
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from skmultilearn.problem_transform import LabelPowerset


import nltk
import glob
import errno
import os
import json

from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import models, corpora, similarities
from gensim.models import CoherenceModel, TfidfModel
from itertools import chain
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer


# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

import pickle


%matplotlib inline

# Load Data

Data contains texts on:
- IPhone X,
    S9,
    Pixel 3,
    Huawei Mate 20 Pro,
    OnePlus 6T,
    Huawei P2 Pro,
    LG V4 Thinq,
    Sony Xperia XZ3,
    Essential Phone,
    Razer Phone 2,
    HTC U12+,
    Moto G6 plus

In [2]:
json_data = pd.DataFrame(columns = ['author', 'text', 'title'])

In [3]:
# Define path to text files that contain the novels
path = '/Users/ayankarim/Documents/Thinkful/Bootcamp/Final Capstone Opinion Mining/opinion_mining/Notebook/Topic Modelling Working Set/files/*.json'
files = glob.glob(path)

all_jsons = []

for filenames in files:
    with open(filenames, 'r') as f:
        file = json.load(f)
        all_jsons.append(file)


In [4]:
for dicts in all_jsons:
    df = pd.DataFrame(dicts)
    json_data = json_data.append(df)

In [5]:
json_data.head()

,author,text,title
0,[\n\t\t\t\t\t\tJulian Chokkattu\t\t\t\t\t],"[<p>Google’s <a href=""https://store.google.com...",\n\t\tGoogle will announce hardware on October...
1,[\n\t\t\t\t\t\tChristian de Looper\t\t\t\t\t],"[<p>Google finally unveiled the new <a href=""h...",\n\t\tHere’s how to buy the new Google Pixel 3...
2,[\n\t\t\t\t\t\tSimon Hill\t\t\t\t\t],[<p>If you plan to buy one of Google’s <a href...,\n\t\tThe best Pixel 3 cases and covers\t
3,[\n\t\t\t\t\t\tSimon Hill\t\t\t\t\t],"[<p>As the developer of Android, Google turns ...",\n\t\tGoogle Pixel 3 vs. Pixel 2 vs. Pixel: Pi...
4,[\n\t\t\t\t\t\tSimon Hill\t\t\t\t\t],[<p>There are plenty of contenders in the <a h...,\n\t\tGoogle Pixel 3 vs. Samsung Galaxy S9: Wh...


# Clean Data

In [6]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!

    text = str(text).replace("\n", "")
    text = str(text).replace("\t", "")
    text = str(text).replace("\\n", "")
    text = str(text).replace("\\t", "")
    text = str(text).replace("\\", "")
    text = str(text).replace("xa0", " ")
    text = str(text).replace("\'", "")
    text = re.sub("<p>", "", str(text))
    text = re.sub("</p>", "", str(text))
    text = re.sub("</a>", "", str(text))
    text = re.sub('<[^>]+>', "", str(text)) 
    text = str(text).replace("\\u2019", "")
    text = str(text).replace("\\u2013", "")
    text = str(text).replace("\\u2018", "")
    text = str(text).replace("\\u00a0", "")
    text = str(text).replace("\\u00a3", "")
    text = str(text).replace("\u2014", "")
    text = str(text).replace("\u201d", "")
    text = str(text).replace("\u201c", "")
    return text


In [7]:
# Define function to clean text
def clean_text(df):
    # Convert lists to strings and remove brackets
    df['text'] = df['text'].astype(str)
    df['author'] = df['author'].astype(str)

    df['text'] = df['text'].map(lambda x: x.strip('[]'))
    df['author'] = df['author'].map(lambda x: x.strip('[]'))

    # Clean text
    df['text'] = df['text'].apply(lambda x: text_cleaner(x))
    df['title'] = df['title'].apply(lambda x: text_cleaner(x))
    df['author'] = df['author'].apply(lambda x: text_cleaner(x))

In [8]:
# Clean Text
clean_text(json_data)

In [9]:
# Reset index
json_data = json_data.reset_index()
json_data = json_data.drop(['index'], axis=1)

In [10]:
# Visualize dataframe
json_data.head()

,author,text,title
0,Julian Chokkattu,Google’s annual hardware launch event will tak...,"Google will announce hardware on October 9, ne..."
1,Christian de Looper,Google finally unveiled the new Google Pixel 3...,Here’s how to buy the new Google Pixel 3 and G...
2,Simon Hill,If you plan to buy one of Google’s Pixel 3 sma...,The best Pixel 3 cases and covers
3,Simon Hill,"As the developer of Android, Google turns out ...",Google Pixel 3 vs. Pixel 2 vs. Pixel: Picking ...
4,Simon Hill,There are plenty of contenders in the Android ...,Google Pixel 3 vs. Samsung Galaxy S9: Which sm...


# Pre-Process Data for NLP

In [11]:

spacy.load('en')
parser = English()

# Function to tokenize text
def tokenize(text):
    lda_tokens = []
    tokens  = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc = True))
        


In [12]:
data = list(json_data['text'])

data_words = list(sent_to_words(data))

In [13]:
print(data_words[:1])

[['google', 'annual', 'hardware', 'launch', 'event', 'will', 'take', 'place', 'on', 'october', 'in', 'new', 'york', 'city', 'the', 'company', 'sent', 'out', 'invites', 'to', 'media', 'including', 'digital', 'trends', 'confirming', 'the', 'date', 'which', 'had', 'leaked', 'in', 'august', 'the', 'date', 'and', 'venue', 'are', 'change', 'of', 'pace', 'considering', 'the', 'past', 'two', 'google', 'october', 'events', 'have', 'taken', 'place', 'in', 'san', 'francisco', 'on', 'october', 'the', 'company', 'is', 'widely', 'expected', 'to', 'launch', 'slew', 'of', 'hardware', 'products', 'ranging', 'from', 'smartphones', 'to', 'smart', 'home', 'devices', 'the', 'highlights', 'will', 'be', 'the', 'pixel', 'and', 'pixel', 'xl', 'successors', 'to', 'last', 'year', 'critically', 'acclaimed', 'pixel', 'and', 'pixel', 'xl', 'smartphones', 'there', 'have', 'been', 'an', 'alarmingly', 'high', 'number', 'of', 'leaks', 'for', 'the', 'pixel', 'series', 'and', 'if', 'true', 'we', 'know', 'quite', 'lot', '

In [14]:
# Create Bigrams and Trigrams

# Build the models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# fast way to get a sentece clubbed as a bigram/trigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [16]:
# Remove Stopwords, make bigrams and lemmatize
stop_words = stopwords.words('english')
stop_words.extend(['pixel', 'iphone', 'samsung', 'apple', 'essential', 'xs', 'max', 
                  'huawei', 'galaxy', 'note', 'moto', 'oneplus', 'android', 'mate', 'pro', 'lg', 'sony', 'razer', 'phone', 'company', 
                  'smartphone', 'google', 'thinq', 'nokia', 'htc', 'xperia', 'xz'])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [17]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['annual', 'hardware', 'launch', 'event', 'take', 'place', 'october', 'new', 'york', 'city', 'send', 'invite', 'medium', 'include', 'digital_trend', 'confirm', 'date', 'leak', 'august', 'date', 'venue', 'change', 'pace', 'consider', 'october', 'event', 'take', 'place', 'san', 'francisco', 'october', 'widely', 'expect', 'launch', 'slay', 'hardware', 'product', 'range', 'smartphone', 'smart', 'home', 'device', 'highlight', 'successor', 'last_year', 'critically', 'acclaim', 'smartphone', 'alarmingly', 'high', 'number', 'leak', 'series', 'true', 'know', 'quite', 'lot', 'phone', 'may', 'due', 'carelessness', 'recently', 'someone', 'leave', 'lyft', 'separately', 'group', 'russia', 'claim', 'get', 'hand', 'shipment', 'smartphone', 'even', 'post', 'unbox', 'video', 'show', 'everything', 'get', 'box', 'expect', 'notch', 'design', 'cutout', 'top', 'screen', 'house', 'front_fac', 'camera', 'garner', 'criticism', 'notch', 'look', 'unusually', 'large', 'small', 'may', 'traditional', 'design', 'sli

In [18]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [19]:
# Feed this data to a pre-built model

In [20]:
# Find another data set to try topic modelling on for comparison

In [21]:
len(corpus[2])

316

In [22]:
# gather tfidf scores
tfidf = models.TfidfModel(corpus, id2word = id2word)

# filter low value words
low_value = 0.025

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    new_bow = [b for b in bow if b[0] not in low_value_words]

    #reassign        
    corpus[i] = new_bow

In [23]:
len(corpus[2])

198

In [24]:
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (5, 1), (6, 3), (7, 1), (8, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 2), (33, 2), (34, 2), (35, 2), (36, 3), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (44, 3), (45, 2), (46, 1), (47, 2), (48, 1), (49, 1), (50, 1), (51, 1), (53, 1), (54, 1), (55, 2), (56, 1), (57, 1), (58, 1), (59, 2), (60, 1), (62, 1), (65, 2), (66, 3), (67, 3), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (75, 1), (78, 1), (81, 2), (82, 1), (83, 4), (85, 1), (87, 1), (88, 2), (89, 1), (90, 1), (91, 2), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 3), (98, 1), (99, 1), (100, 1), (101, 1), (102, 3), (103, 1), (104, 1), (105, 1), (106, 1), (107, 2), (108, 1), (109, 1), (110, 1), (112, 4), (113, 3), (114, 1), (115, 1), (116, 1), (117, 1), (118, 1), (119, 1), (120, 1), (122, 1), (123, 2), (124, 1), (125, 1), (126, 1), (127, 1),

In [25]:
# Build the lda topic model

lda = models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, update_every=1, 
                               chunksize=50, passes=25, random_state=1, alpha='auto', minimum_probability=0)

# save the model to disk
# filename = 'lda_model.pkl'
# pickle.dump(lda, open(filename, 'wb'))

1. topic 0
    - camera, display, feature, megapixel, lens, screen, big, device
    
    reliability
    
2. topic 1
    - screen, app, case, tap, go, option, display, setting, button, notification, select
    
    functionality
    
3. topic 2
    - camera, photo, device, work, user, issue, leak, show, app, really, update
    
    design

In [26]:
# Print the key words in the 10 topics
pprint(lda.print_topics(num_words=15))
doc_lda = lda[corpus]

[(0,
  '0.018*"camera" + 0.013*"phone" + 0.008*"good" + 0.008*"display" + '
  '0.008*"feature" + 0.007*"lens" + 0.007*"megapixel" + 0.006*"offer" + '
  '0.006*"winner" + 0.006*"find" + 0.006*"screen" + 0.005*"get" + '
  '0.005*"device" + 0.005*"come" + 0.005*"update"'),
 (1,
  '0.017*"app" + 0.017*"screen" + 0.014*"case" + 0.011*"tap" + 0.008*"home" + '
  '0.007*"set" + 0.006*"want" + 0.006*"display" + 0.006*"setting" + '
  '0.006*"option" + 0.006*"go" + 0.006*"turn" + 0.006*"button" + 0.005*"use" + '
  '0.005*"time"'),
 (2,
  '0.007*"photo" + 0.006*"camera" + 0.006*"even" + 0.006*"device" + '
  '0.004*"could" + 0.004*"say" + 0.004*"take" + 0.004*"issue" + 0.004*"user" + '
  '0.004*"really" + 0.004*"leak" + 0.004*"thing" + 0.003*"people" + '
  '0.003*"seem" + 0.003*"new"')]


In [27]:
# Evaluate how good the model is

# Compute perplexity
print('\nPerplexity: ', lda.log_perplexity(corpus)) # Lower the better

# Compute Coherence
coherenece_model_lda = CoherenceModel(model=lda, texts=data_lemmatized, dictionary=id2word, coherence='c_v' )
coherence_lda = coherenece_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.7599402552235945

Coherence Score:  0.3891545948178223


In [28]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, id2word)
vis

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.073660  0.151508       1        1  52.247585
2     -0.120846 -0.128838       2        1  30.573595
1      0.194506 -0.022670       3        1  17.178818, topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
9     Default  345.000000           app  345.000000  30.0000  30.0000
102   Default  513.000000        screen  513.000000  29.0000  29.0000
86    Default  552.000000         phone  552.000000  28.0000  28.0000
145   Default  243.000000          case  243.000000  27.0000  27.0000
658   Default  180.000000           tap  180.000000  26.0000  26.0000
568   Default  300.000000     megapixel  300.000000  25.0000  25.0000
337   Default  303.000000          lens  303.000000  24.0000  24.0000
193   Default  121.000000           set  121.000000  23.0000  23.0000
59    Default  166.000000          home  166.000000  22.0000  22.0000
675   Default  266.000000        winner  266.000000  21.0000  21.0000
362   Default  153.000000        option  153.000000  20.0000  20.0000
981   Default  104.000000       setting  104.000000  19.0000  19.0000
464   Default  179.000000           use  179.000000  18.0000  18.0000
158   Default  378.000000       feature  378.000000  17.0000  17.0000
127   Default  109.000000          turn  109.000000  16.0000  16.0000
43    Default  219.000000          even  219.000000  15.0000  15.0000
469   Default  213.000000          want  213.000000  14.0000  14.0000
25    Default  211.000000          come  211.000000  13.0000  13.0000
581   Default  263.000000         photo  263.000000  12.0000  12.0000
488   Default  208.000000       battery  208.000000  11.0000  11.0000
253   Default  123.000000         could  123.000000  10.0000  10.0000
308   Default  164.000000            go  164.000000   9.0000   9.0000
223   Default  195.000000          back  195.000000   8.0000   8.0000
84    Default  284.000000         offer  284.000000   7.0000   7.0000
949   Default   75.000000  notification   75.000000   6.0000   6.0000
238   Default  162.000000        button  162.000000   5.0000   5.0000
309   Default  416.000000          good  416.000000   4.0000   4.0000
326   Default  183.000000          inch  183.000000   3.0000   3.0000
36    Default  487.000000       display  487.000000   2.0000   2.0000
67    Default   95.000000          leak   95.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
145    Topic3  194.158920          case  243.903458   1.5334  -4.3033
193    Topic3   96.453339           set  121.030563   1.5345  -5.0029
9      Topic3  249.321472           app  345.417542   1.4355  -4.0532
1227   Topic3   21.323788          wake   23.578533   1.6610  -6.5122
127    Topic3   81.465889          turn  109.225937   1.4683  -5.1718
388    Topic3   54.012352    protection   70.915504   1.4892  -5.5828
59     Topic3  109.466728          home  166.509689   1.3421  -4.8764
102    Topic3  241.881485        screen  513.298828   1.0091  -4.0835
575    Topic3   50.640160          open   71.634216   1.4147  -5.6472
989    Topic3   26.277370        slider   31.171520   1.5907  -6.3033
362    Topic3   86.477066        option  153.295837   1.1890  -5.1121
387    Topic3   31.585155       protect   40.614304   1.5101  -6.1193
308    Topic3   83.178963            go  164.002594   1.0826  -5.1510
480    Topic3   58.666977         allow   99.479736   1.2334  -5.5001
238    Topic3   79.746689        button  162.608139   1.0490  -5.1931
3218   Topic3   34.296307      keyboard   46.630650   1.4543  -6.0369
505    Topic3   46.550537        choose   74.305725   1.2938  -5.7314
199    Topic3   73.694450          time  152.394165   1.0350  -5.2721
469    Topic3   90.147858          want  213.405319   0.8998  -5.0705
464    Topic3   77.960960           use  179.52

In [29]:
# Mallet LDA

mallet_path = '/Users/ayankarim/mallet-2.0.8/bin/mallet'

ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=3, id2word=id2word, iterations=1000, random_seed=100 )

1. topic 0
    - device, photo, year, issue, big, people, buy, price, user, mobile
    
    reliability
    
2. topic 1
    - camera, feature, lens, megapixel, display, update, design, play
    
    design
    
3. topic 2
    - screen, app, case, mode, tap, button, display, option, home, time, battery, setting, light
    
    functionality

In [30]:
# Show topics
pprint(ldamallet.show_topics(num_words=15, formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(0,
  [('device', 0.012748117727583847),
   ('photo', 0.010438056125941135),
   ('thing', 0.007486310746064339),
   ('year', 0.006930184804928132),
   ('good', 0.006160164271047228),
   ('issue', 0.00607460643394935),
   ('big', 0.006031827515400411),
   ('people', 0.005946269678302533),
   ('buy', 0.005775154004106776),
   ('feel', 0.005775154004106776),
   ('price', 0.0053901437371663245),
   ('user', 0.0053901437371663245),
   ('mobile', 0.005176249144421629),
   ('rear', 0.005047912388774812),
   ('handset', 0.004791238877481177)]),
 (1,
  [('camera', 0.036118409528814156),
   ('phone', 0.02161499386932913),
   ('feature', 0.012401471361008934),
   ('lens', 0.011770888071466106),
   ('good', 0.010965142757050272),
   ('megapixel', 0.010895077947101069),
   ('display', 0.010825013137151866),
   ('offer', 0.010509721492380452),
   ('find', 0.010404624277456647),
   ('winner', 0.009143457698370994),
   ('update', 0.008302679978980557),
   ('back', 0.007286740234717114),
   ('inch', 0

In [31]:
# Assigns the topics to the documents in corpus
lda_corpus = lda[corpus]

# Find the threshold, let's set the threshold to be 1/#clusters,
# To prove that the threshold is sane, we average the sum of all probabilities:
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))
threshold = sum(scores)/len(scores)
print (threshold)

reliability = [j for i,j in zip(lda_corpus,data) if i[0][1] > threshold]
functionality = [j for i,j in zip(lda_corpus,data) if i[1][1] > threshold]
design = [j for i,j in zip(lda_corpus,data) if i[2][1] > threshold]

0.3333333332180924


In [32]:
# Create dataframe of document groups for topics
topics_df = pd.DataFrame(reliability, columns=['reliability'])
topics_df['functionality'] = pd.DataFrame(functionality)
topics_df['design'] = pd.DataFrame(design)

In [33]:
topics_df.head()

,reliability,functionality,design
0,Google finally unveiled the new Google Pixel 3...,If you plan to buy one of Google’s Pixel 3 sma...,Google’s annual hardware launch event will tak...
1,"As the developer of Android, Google turns out ...",Got your hands on a new Pixel 3 or Pixel 3 XL ...,The Google Pixel 3 and Pixel 3 XL are arguably...
2,There are plenty of contenders in the Android ...,The Google Pixel 3 and Pixel 3 XL may have sto...,The Google Pixel 2 and Pixel 2 XL may be the c...
3,Rarely has a flagship phone been so thoroughly...,The Google Pixel 3 and Pixel 3 XL are phones w...,The Google Pixel 3 and Pixel 3 XL may already ...
4,"Mobile hardware is getting better and better, ...","After months of rumors and leaks, Google is fi...","After months of rumors and leaks, Google is fi..."


In [34]:
labelled_df = json_data

# Assign labels as binary topics
labelled_df['reliability'] = ""
labelled_df['functionality'] = ""
labelled_df['design'] = ""

labelled_df['reliability'] = np.where(labelled_df['text'].isin(reliability), 'reliability', None)
labelled_df['functionality'] = np.where(labelled_df['text'].isin(functionality), 'functionality', None)
labelled_df['design'] = np.where(labelled_df['text'].isin(design), 'design', None)

In [35]:
labelled_df.head()

,author,text,title,reliability,functionality,design
0,Julian Chokkattu,Google’s annual hardware launch event will tak...,"Google will announce hardware on October 9, ne...",None,None,design
1,Christian de Looper,Google finally unveiled the new Google Pixel 3...,Here’s how to buy the new Google Pixel 3 and G...,reliability,None,None
2,Simon Hill,If you plan to buy one of Google’s Pixel 3 sma...,The best Pixel 3 cases and covers,None,functionality,None
3,Simon Hill,"As the developer of Android, Google turns out ...",Google Pixel 3 vs. Pixel 2 vs. Pixel: Picking ...,reliability,None,None
4,Simon Hill,There are plenty of contenders in the Android ...,Google Pixel 3 vs. Samsung Galaxy S9: Which sm...,reliability,None,None


In [36]:
labelled_df['labelled'] = labelled_df[['reliability', 'functionality', 'design']].values.tolist()
labelled_df['labelled'] = labelled_df['labelled'].apply(lambda x: list(filter(lambda a: a != None, x)))
labelled_df = labelled_df.drop(['reliability', 'functionality', 'design'], axis=1)

In [37]:
labelled_df.head()

,author,text,title,labelled
0,Julian Chokkattu,Google’s annual hardware launch event will tak...,"Google will announce hardware on October 9, ne...",[design]
1,Christian de Looper,Google finally unveiled the new Google Pixel 3...,Here’s how to buy the new Google Pixel 3 and G...,[reliability]
2,Simon Hill,If you plan to buy one of Google’s Pixel 3 sma...,The best Pixel 3 cases and covers,[functionality]
3,Simon Hill,"As the developer of Android, Google turns out ...",Google Pixel 3 vs. Pixel 2 vs. Pixel: Picking ...,[reliability]
4,Simon Hill,There are plenty of contenders in the Android ...,Google Pixel 3 vs. Samsung Galaxy S9: Which sm...,[reliability]


In [38]:
# Convert the multi-labels into arrays
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(labelled_df.labelled)
X = labelled_df.text

# Split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# save the the fitted binarizer labels
# This is important: it contains the how the multi-label was binarized, so you need to
# load this in the next folder in order to undo the transformation for the correct labels.
# filename = 'mlb.pkl'
# pickle.dump(mlb, open(filename, 'wb'))

In [39]:
# LabelPowerset allows for multi-label classification
# Build a pipeline for multinomial naive bayes classification
text_clf = Pipeline([('vect', CountVectorizer(stop_words = "english",ngram_range=(1, 1))),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     ('clf', LabelPowerset(MultinomialNB(alpha=1e-1))),])

# 'vect', CountVectorizer(stop_words = "english",ngram_range=(1, 1))),

In [40]:
text_clf = text_clf.fit(X_train, y_train)
predicted = text_clf.predict(X_test)

# Calculate accuracy
np.mean(predicted == y_test)

0.8148148148148148

In [41]:
# Test if SVM performs better

text_clf_svm = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', LabelPowerset(
                             SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, max_iter=6, random_state=42)))])
_ = text_clf_svm.fit(X_train, y_train)
predicted_svm = text_clf_svm.predict(X_test)

#Calculate accuracy
np.mean(predicted_svm == y_test)

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


0.875

In [42]:
# Train naive bayes on full dataset and save model
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words = "english",ngram_range=(1, 1))),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     ('clf', LabelPowerset(MultinomialNB(alpha=1e-1))),])
text_clf_svm = text_clf.fit(X, y)

# save the model to disk
# filename = 'svm_model.pkl'
# pickle.dump(text_clf_svm, open(filename, 'wb'))

In [43]:
## Train new data and labels with multi-label naive bayes
## Evaluate Model on test set


## Save lda model to assign topics in Opinion Mining notebook
## Save ML-NB model to assign aspect terms to topics in Opinion Mining notebook

In [44]:
## Pickle dump the topic models
## Call the models the in the opinion mining notebook and use them to classify the texts into topics
## Use Multilabel Naive Bayes to get an output of aspect terms in a dataframe grouped into topic columns, ordered by score

### HOW TO EVALUATE YOUR MODEL!!??

## Clean the Notebook, write up a report
## Create Presentation